In [1]:
from restCall import RestCall


In [5]:
rest = RestCall("https://rpa.casthighlight.com/WS2","n.kaplan+WellsFargo@castsoftware.com","mdSi20ty@02",False)
(status,data) = rest.get("/domains/1271/applications")
 

In [7]:
import pandas as pd
apps = pd.DataFrame(data)
apps = apps[['id','clientRef','name']]

In [11]:
import util
file_name = f'../data/hl_apps.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
col_widths=[10,50,50]
util.format_table(writer,apps,'Existing',col_widths)
writer.save()
